# 2.2 SOO for 1D scalars

In [1]:
%matplotlib inline
import sys
import os

# Sort the paths out to run from this file
parent_path = os.path.dirname(os.getcwd())
src_path = os.path.join(os.path.sep,parent_path, 'src')
sys.path.append(src_path)
print(src_path)

/home/luke/code/cyclops/src


We instantiate the objects we need (change the temperature field to a different field if you have saved your field with a different name in steps 1 and 2). We also set some bounds for the sensors to go in.

In [2]:
from run_me import *


# Load any objects necessary
pickle_manager = PickleManager()
graph_manager = PlotManager()
true_temp_field = pickle_manager.read_file('simulation', 'temp_line_field.pickle')
grid = pickle_manager.read_file('simulation', 'temp_line_points.pickle')

field_bounds = true_temp_field.get_bounds()
sensor_bounds = field_bounds+np.array([[1], [-1]])*0.002

ModuleNotFoundError: No module named 'run_me'

We define the symmetry assumptions we want to force (like in manual 2.1) and the kinds of sensor we want to use. Uncomment the code below to see how you can setup a point sensor or a multi sensor (or use a combination of these). The multisensor is meant to behave like an IR camera or a DIC if you define the systematic error function well enough.

In [ ]:
# Setup the symmetry
symmetry_manager = SymmetryManager()
symmetry_manager.set_1D_x(0.01)

# Setup the sensor suite
temps = pickle_manager.read_file('sensors', 'k-type-T.pickle')
voltages = pickle_manager.read_file('sensors', 'k-type-V.pickle')
sensor = Thermocouple(temps, voltages, 1)
sensors = np.array([sensor]*5)

# def f(x): return np.zeros(x.shape)
# sensor = PointSensor(0, f, 0.1, np.array([[-5e10], [5e10]]), 1)
# sensors = np.array([sensor]*5)


# def f(x): return np.zeros(x.shape)
# sensor_1 = MultiSensor(0, f, 0.1, np.array([[-5000], [5000]]), np.linspace(sensor_bounds[0, 0], sensor_bounds[1, 0], 10).reshape(-1, 1))
# sensors = np.array([sensor_1, sensor, sensor, sensor])

sensor_suite = SensorSuite(
    ScalarField(CSModel, field_bounds), 
    sensors
)

We setup the optimisation and optimise as in 2.1. Remember to look at the optional parameters.

In [ ]:
# Setup the experiment
optimiser = PSOOptimiser('00:00:10')
experiment = Experiment(
    true_temp_field,
    grid,
    optimiser
)
experiment.plan_soo(
    sensor_suite,
    sensor_bounds
)
res = experiment.design()
proposed_layout, true_temps, model_temps, sensor_values = experiment.get_SOO_plotting_arrays(res.X)

We look at the results.

In [ ]:
# Display the results
graph_manager.build_optimisation(
    res.history
)
graph_manager.draw()
graph_manager.build_1D_compare(
    grid,
    proposed_layout,
    sensor_values,
    true_temps,
    model_temps
)
graph_manager.draw()